# Genetic Toggle Switch
***
Gardner et al. Nature (1999) 'Construction of a genetic toggle switch in Escherichia coli'
***
## Setup the Environment
***

In [ ]:
import numpy
from dask.distributed import Client

In [ ]:
import gillespy2

In [ ]:
from sciope.utilities.gillespy2 import wrapper
from sciope.designs import latin_hypercube_sampling
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH
from sciope.stochmet.stochmet import StochMET
from sciope.models.label_propagation import LPModel

***
## Create the Genetic Toggle Switch Model
***

In [ ]:
def create_genetic_toggle_switch(parameter_values=None):
    model = gillespy2.Model(name="Toggle_Switch")
    model.volume = 1

    # Variables
    U = gillespy2.Species(name="U", initial_value=10, mode="discrete")
    V = gillespy2.Species(name="V", initial_value=10, mode="discrete")
    model.add_species([U, V])

    # Parameters
    alpha1 = gillespy2.Parameter(name="alpha1", expression="1")
    alpha2 = gillespy2.Parameter(name="alpha2", expression="1")
    beta = gillespy2.Parameter(name="beta", expression="2")
    gamma = gillespy2.Parameter(name="gamma", expression="2")
    mu = gillespy2.Parameter(name="mu", expression="1")
    model.add_parameter([alpha1, alpha2, beta, gamma, mu])

    # Reactions
    cu = gillespy2.Reaction(
        name="cu",
        reactants={}, products={'U': 1},
        propensity_function="alpha1/(1+pow(V,beta))",
        ode_propensity_function="alpha1/(1+pow(V,beta))"
    )
    cv = gillespy2.Reaction(
        name="cv",
        reactants={}, products={'V': 1},
        propensity_function="alpha2/(1+pow(U,gamma))",
        ode_propensity_function="alpha2/(1+pow(U,gamma))"
    )
    du = gillespy2.Reaction(
        name="du", rate="mu",
        reactants={'U': 1}, products={}
    )
    dv = gillespy2.Reaction(
        name="dv", rate="mu",
        reactants={'V': 1}, products={}
    )
    model.add_reaction([cu, cv, du, dv])

    # Timespan
    tspan = gillespy2.TimeSpan.arange(1, t=101)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_genetic_toggle_switch()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    solver = gillespy2.SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

***
## Model Exploration
***
### Define simulator function (using gillespy2 wrapper)

In [ ]:
settings = configure_simulation()
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['U', 'V'])
expression_array = wrapper.get_parameter_expression_array(model)

### Start local cluster using dask client

In [ ]:
c = Client()
c

### Define parameter sampler/design and summary statistics

In [ ]:
lhc = latin_hypercube_sampling.LatinHypercube(xmin=expression_array, xmax=expression_array*3)
lhc.generate_array(1000) # creates a LHD of size 1000

# will use default minimal set of features
summary_stats = SummariesTSFRESH()

### Initiate StochMET

In [ ]:
met = StochMET(simulator, lhc, summary_stats)

***
## Run parameter sweep
***

In [ ]:
met.compute(n_points=500, chunk_size=10)

***
## Explore the result
***
First lets add some appropiate information about the model and features

In [ ]:
met.data.configurations['listOfParameters'] = list(model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = model.tspan

Here we use UMAP for dimension reduction

In [ ]:
met.explore(dr_method='umap')

Here lets use the dimension reduction embedding as input data

In [ ]:
data = met.dr_model.embedding_

model_lp = LPModel()
# train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

Just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model output when using method 'explore')

In [ ]:
user_labels = numpy.copy(met.data.user_labels)
# takes the label corresponding to index 0
met.data.user_labels = model_lp.model.label_distributions_[:, 0]

In [ ]:
met.explore(dr_method='umap')

In [ ]:
met.data.user_labels = user_labels